<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Obtaining-Your-Data---Lab" data-toc-modified-id="Obtaining-Your-Data---Lab-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Obtaining Your Data - Lab</a></span><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Task-Description" data-toc-modified-id="Task-Description-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Task Description</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Summary</a></span></li></ul></li></ul></div>

# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Perform an ETL process with multiple tables and create a single dataset

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this SQL query to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
            WHERE type='table'
            ORDER BY name;
```

In [6]:
# Your code here
import sqlite3
import pandas as pd

con = sqlite3.connect('lego.db')
cur = con.cursor()
cur.execute("""SELECT * 
               FROM sqlite_master 
               WHERE type = 'table'
               ORDER BY name
               ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(df.shape)
df.head()

(4, 5)


,type,name,tbl_name,rootpage,sql
0,table,product_details,product_details,5,CREATE TABLE product_details (prod_id INTEGER ...
1,table,product_info,product_info,2,CREATE TABLE product_info (prod_id INTEGER PRI...
2,table,product_pricing,product_pricing,3,CREATE TABLE product_pricing (prod_id INTEGER ...
3,table,product_reviews,product_reviews,6,CREATE TABLE product_reviews (prod_id INTEGER ...


4 tables, prod_id is primary key?

In [8]:
cur.execute("""SELECT * 
               FROM product_details
               ;""")

df1 = pd.DataFrame(cur.fetchall())
df1.columns = [i[0] for i in cur.description]
df1.head()

,prod_id,prod_desc,prod_long_desc,theme_name
0,630,blah,blahblah,Classic
1,2304,blah,blahblah,DUPLO®
2,7280,blah,blahblah,City
3,7281,blah,blahblah,City
4,7499,blah,blahblah,City


prod_id: 630, 2304

In [10]:
cur.execute("""SELECT * 
               FROM product_info
               ;""")

df2 = pd.DataFrame(cur.fetchall())
df2.columns = [i[0] for i in cur.description]
df2.head()

,prod_id,ages,piece_count,set_name
0,630,4+,1,Brick Separator
1,2304,1½-5,1,LEGO® DUPLO® Green Baseplate
2,7280,5-12,2,Straight & Crossroad Plates
3,7281,5-12,2,T-Junction & Curved Road Plates
4,7499,5-12,24,Flexible and Straight Tracks


In [11]:
cur.execute("""SELECT * 
               FROM product_pricing
               ;""")

df3 = pd.DataFrame(cur.fetchall())
df3.columns = [i[0] for i in cur.description]
df3.head()

,prod_id,country,list_price
0,75823,US,29.99
1,75822,US,19.99
2,75821,US,12.99
3,21030,US,99.99
4,21035,US,79.99


In [12]:
cur.execute("""SELECT * 
               FROM product_reviews
               ;""")

df4 = pd.DataFrame(cur.fetchall())
df4.columns = [i[0] for i in cur.description]
df4.head()

,prod_id,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,630,180.0,4.0,Very Easy,4.8,4.6
1,2304,15.0,4.4,Easy,4.0,3.3
2,7280,85.0,4.1,Very Easy,3.5,2.3
3,7281,40.0,4.1,Very Easy,3.9,2.8
4,7499,89.0,2.9,Easy,2.5,2.2


In [15]:
cur.execute("""SELECT * 
               FROM product_details 
               JOIN product_info 
               USING(prod_id)
               JOIN product_pricing 
               USING(prod_id)
               JOIN product_reviews 
               USING(prod_id)
               ;""")

lego_df = pd.DataFrame(cur.fetchall())
lego_df.columns = [i[0] for i in cur.description]
lego_df.head()

,prod_id,prod_desc,prod_long_desc,theme_name,ages,piece_count,set_name,country,list_price,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,blah,blahblah,Angry Birds™,6-12,277,Bird Island Egg Heist,US,29.99,2.0,4.0,Average,4.5,4.0
1,75822,blah,blahblah,Angry Birds™,6-12,168,Piggy Plane Attack,US,19.99,2.0,4.0,Easy,5.0,4.0
2,75821,blah,blahblah,Angry Birds™,6-12,74,Piggy Car Escape,US,12.99,11.0,4.3,Easy,4.3,4.1
3,21030,blah,blahblah,Architecture,12+,1032,United States Capitol Building,US,99.99,23.0,3.6,Average,4.6,4.3
4,21035,blah,blahblah,Architecture,12+,744,Solomon R. Guggenheim Museum®,US,79.99,14.0,3.2,Challenging,4.6,4.1


In [16]:
#Answer key solution to preview the tables:
# conn = sqlite3.connect('lego.db')
# c = conn.cursor()
# response = c.execute("""SELECT name FROM sqlite_master
#              WHERE type='table'
#              ORDER BY name;
#           """).fetchall()
# response

# for item in response:
#     table = item[0]
#     length = c.execute("""SELECT count(*) from {};""".format(table)).fetchall()
#     results = c.execute("""SELECT * from {} limit 5;""".format(table)).fetchall()
#     df = pd.DataFrame(results)
#     df.columns = [x[0] for x in c.description]
#     print(table, length, '\n', df, '\n\n')

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity! In this lab, you successfully executed an ETL process to merge different tables!